# Gotobi Strategy Backtest Demo

Demonstrates running the Gotobi FX settlement strategy using NautilusTrader's BacktestEngine.
Loads USDJPY data from parquet files and runs both the basic Gotobi and stop-loss variants.

In [ ]:
import sys, os
sys.path.append(os.path.abspath("../.."))

import pandas as pd
import plotly.graph_objects as go

from nautilus_trader.model.data import BarType
from nautilus_trader.model.enums import AccountType, OmsType

from trader import (
    DataHandler,
    make_fx_pair,
    build_backtest_engine,
    GotobiStrategy,
    GotobiConfig,
    GotobiWithSLStrategy,
    GotobiWithSLConfig,
    SIM,
)
from trader.data.catalog import dataframe_to_nautilus_bars

## Load Data

In [ ]:
# Load USDJPY parquet data
handler = DataHandler()
df = handler.load_parquet("../../data/USDJPY_1min_2024-01-01_2025-10-01.parquet")

# Convert timezone for Japanese trading hours
df.index = df.index.tz_convert("Asia/Tokyo")
print(f"Loaded {len(df)} bars from {df.index[0]} to {df.index[-1]}")

## Build Instruments and Bar Data

In [ ]:
# Create USDJPY instrument
instrument = make_fx_pair("USDJPY", SIM, lot_size=100_000)

# Define bar type
bar_type = BarType.from_str(f"{instrument.id}-1-MINUTE-MID-EXTERNAL")

# Convert DataFrame to NautilusTrader bars
bars = dataframe_to_nautilus_bars(df, bar_type)
print(f"Converted {len(bars)} bars for {bar_type}")

## Run Gotobi Strategy (No Stop Loss)

In [ ]:
# Configure strategies
long_config = GotobiConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="09:00:00",
    exit_time="09:55:00",
    trade_size=50.0,
    contract_size=1.0,  # already sized in lots
    use_holidays=True,
)

short_config = GotobiConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="10:00:00",
    exit_time="10:45:00",
    trade_size=-50.0,
    contract_size=1.0,
    use_holidays=True,
)

long_strat = GotobiStrategy(config=long_config)
short_strat = GotobiStrategy(config=short_config)

# Build and run engine
engine = build_backtest_engine(
    instruments=[instrument],
    bars={bar_type: bars},
    strategies=[long_strat, short_strat],
    venue=SIM,
    venue_currency="USD",
    starting_balance=1000,
    leverage=500.0,
)

engine.run()
print(f"Backtest complete. Final account value available in engine reports.")

## Run Gotobi With Stop Loss

In [ ]:
# Reset and run with stop-loss variant
long_sl_config = GotobiWithSLConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="09:00:00",
    exit_time="09:55:00",
    trade_size=50.0,
    contract_size=1.0,
    stop_loss_pct=0.0005,
    use_holidays=True,
)

short_sl_config = GotobiWithSLConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="10:00:00",
    exit_time="10:45:00",
    trade_size=-50.0,
    contract_size=1.0,
    stop_loss_pct=0.0005,
    use_holidays=True,
)

long_sl = GotobiWithSLStrategy(config=long_sl_config)
short_sl = GotobiWithSLStrategy(config=short_sl_config)

engine_sl = build_backtest_engine(
    instruments=[instrument],
    bars={bar_type: bars},
    strategies=[long_sl, short_sl],
    venue=SIM,
    venue_currency="USD",
    starting_balance=1000,
    leverage=500.0,
)

engine_sl.run()
print("Stop-loss backtest complete.")